Click the play button in the cell below to load all tools we'll need. You'll be asked for your list of MMSIDs.

In [19]:
import os
from IPython.display import display
import ipywidgets as widgets

def clicked_sand(arg):
    apiKey = os.environ['API_KEY_SAND']
    with out:
        print('\rusing Sandbox data   ', end='')

def clicked_prod(arg):
    apiKey = os.environ['API_KEY_PROD']
    with out:
        print('\rusing Production data', end='')

print('Would you like to use the Sandbox or Production environment?')
button_sand = widgets.Button(description = 'Sandbox', tooltip='Sandbox')   
button_sand.on_click(clicked_sand)

button_prod = widgets.Button(description = 'Production', tooltip='Production')   
button_prod.on_click(clicked_prod)

out = widgets.Output()
buttons = widgets.HBox([button_sand, button_prod])
outdisplay = widgets.VBox([out])
display(buttons, outdisplay)



Would you like to use the Sandbox or Production environment?
